In [1]:
import json
import pandas as pd
import os
import numpy as np
import scipy.signal as sig
from biosppy.signals.bvp import bvp
from biosppy.signals.eda import eda
import matplotlib.pyplot as plt
import scipy.io as sio
import pickle
import sklearn
from sklearn.model_selection import train_test_split
from scipy.signal import butter, lfilter, iirnotch, welch

%matplotlib qt

### Data Preparation

![dataset_sampling_rates.png](Images/dataset_sampling_rates.png)

Notes About Signal Preparation:
- All Empatica Signals were upsampled to match the muse samples so that the data can be inputted into a CNN model consistently. This means that the sampling frequency used accross those signals is 256 Hz.
- Samsung watch signals not included at the moment because they have a sampling rate that is inconsistent with the Muse Headset
- Important modifications before inputing data to model:
    - Make sure to center at zero
    - Standardize data when appropriate (Such as for filtered EEG data)
    - Use timestamps to allign to stimuli but do not input it into model
    - Data augmentations to keep in mind: flipping the signal, making the overlap between signals larger
    - Possibly downsampling all the signals so that it is easier for the CNN to find more low frequency patterns (as may be expected to exist in signals for emotion recognition)
        - To this end, also if wanting to use power spectrum rather than raw EEG signal, this would make sense

In [3]:
with open(r"C:\Users\nelso\Desktop\18065 Project Data\Emognition\22\22_AMUSEMENT_STIMULUS_MUSE.json") as json_file:
    data = json.load(json_file)

In [74]:
def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    '''
    Performs bandpass filter on EEG data

    data: array, of shape channels by samples
    lowcut: int, the lower cut-off frequency
    highcut: int, the higher cut-off frequency
    fs: int, sampling frequency of the signal
    order: int, the order of the butterworth filter

    return: array, of same shape as data with filtered data
    '''
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    y = lfilter(b, a, data)
    return y

def norm_stand_data(X,max_val=1,min_val=0):
    '''
    Standardizes and normalizes data

    X: array, of shape trials by channels by samples
    max_val: float, the maximum value to standardize data to
    min_val: float, the minimum value to standardize data to

    return: array, of the same shape as X but having standardized values
    '''
    X_std = (X - X.min(axis=1,keepdims=True)) / (X.max(axis=1,keepdims=True) - X.min(axis=1,keepdims=True))
    X_scaled = X_std * (max_val - min_val) + min_val
    return X_scaled

In [38]:
def eeg_analysis(eeg_data,l_freq=5,h_freq=55):
    eeg_df = pd.DataFrame.from_dict(eeg_data)
    eeg_timestamp = eeg_df.pop("TimeStamp")
    eeg_df = eeg_df.astype("float32")
    eeg_df.drop(['Delta_TP9', 'Delta_AF7', 'Delta_AF8', 'Delta_TP10', 'Theta_TP9',
               'Theta_AF7', 'Theta_AF8', 'Theta_TP10', 'Alpha_TP9', 'Alpha_AF7',
               'Alpha_AF8', 'Alpha_TP10', 'Beta_TP9', 'Beta_AF7', 'Beta_AF8',
               'Beta_TP10', 'Gamma_TP9', 'Gamma_AF7', 'Gamma_AF8', 'Gamma_TP10'],axis=1,inplace=True)
    eeg_df = eeg_df.interpolate()
    eeg_df["TimeStamp"] = eeg_timestamp
    for eeg_chan in ['RAW_TP9', 'RAW_AF7', 'RAW_AF8', 'RAW_TP10']:
        filtered_eeg = butter_bandpass_filter(eeg_df[eeg_chan].to_numpy(), l_freq, h_freq, 256, order=4)
        # print("filtered_eeg shape: ", filtered_eeg.shape)
        # print(filtered_eeg)
        eeg_df["FILT_"+eeg_chan.split("_")[1]] = filtered_eeg
    eeg_sig_len = len(eeg_df[:])
    return eeg_df[:], eeg_sig_len

def bvp_analysis(data,eeg_len,do_plot = False):
    bvp_data = np.array(data['BVP'])[:,-1].astype('float32')
    bvp_data = sig.resample(bvp_data,eeg_len)
    _, _, _, _, heart_rate = bvp(bvp_data,256,show=do_plot)
    heart_rate = sig.resample(heart_rate,eeg_len)

    if do_plot:
        plt.figure()
        plt.plot([i/256 for i in range(len(heart_rate))],heart_rate)
        plt.title("Heart Rate Interpolated")
    return bvp_data,heart_rate

# EDA Signal Analysis
def eda_analysis(data,eeg_len,do_plot = False):
    eda_data = np.array(data['EDA'])[:,-1].astype('float32')

    eda_data = sig.resample(eda_data,eeg_len)
    _, filtered_eda, _, _, _ = eda(eda_data,256,show=do_plot)
    return filtered_eda

# TESTING

# To test eeg_analysis data frame generation
with open(r"C:\Users\nelso\Desktop\18065 Project Data\Emognition\22\22_AMUSEMENT_STIMULUS_MUSE.json") as json_file:
    eeg_data = json.load(json_file)
    eeg_df, eeg_sig_len = eeg_analysis(eeg_data)

# To test other physiolocial signals processing
with open(r"C:\Users\nelso\Desktop\18065 Project Data\Emognition\22\22_AMUSEMENT_STIMULUS_EMPATICA.json") as json_file:
    data = json.load(json_file)
#BVP signal analysis:

bvp_analysis(data,eeg_sig_len,True)

eda_analysis(data,eeg_sig_len,True)

array([0.28718999, 0.28714483, 0.28709843, ..., 0.27497005, 0.2750411 ,
       0.27511047])

In [31]:
file_loc = r"C:\Users\nelso\Desktop\18065 Project Data\Emognition"

In [39]:
emotions = ["BASELINE","NEUTRAL","AMUSEMENT","ANGER","AWE","DISGUST","ENTHUSIASM","FEAR","LIKING","SADNESS","SURPRISE"]
class DeviceMissing(Exception):
    """Device Missing Exception"""
    pass

devices_of_interest = ["MUSE","EMPATICA"] #MUSE must be loaded first!
Testing = False
per_user_labels = {}
per_user_stimulus_arr = {}
for root,dir,_ in os.walk(file_loc):
    for d in dir:
        participant = os.path.join(root,d)

        #Map user to Label in Dicionary
        emotion_dict = {e:{} for e in emotions}
        labels_loc = participant + "\\" + d + "_QUESTIONNAIRES.json"
        with open(labels_loc) as json_file:
            quest_results = json.load(json_file)
            for condition in quest_results['questionnaires']:
                cond_type = condition["movie"]
                emotion_dict[cond_type] = condition["emotions"]
        per_user_labels[d] = emotion_dict
        per_user_stimulus_arr[d] = {}

        for emotion in emotions:
            for device in devices_of_interest:
                data_loc = participant + f"\\{d}_{emotion}_STIMULUS_{device}.json"

                try:
                    with open(data_loc) as json_file:
                        data = json.load(json_file)
                        if device == "MUSE":
                            per_user_stimulus_arr[d][emotion],eeg_sig_len = eeg_analysis(data)
                        elif device == "EMPATICA":
                            bvp_data, heart_rate = bvp_analysis(data,eeg_sig_len,do_plot=Testing)
                            per_user_stimulus_arr[d][emotion]["BVP_Filt"] = bvp_data
                            per_user_stimulus_arr[d][emotion]["HR_EMPATICA"] = heart_rate
                            filt_eda = eda_analysis(data,eeg_sig_len,do_plot = Testing)
                            per_user_stimulus_arr[d][emotion]["EDA_EMPATICA"] = filt_eda
                        else:
                            raise DeviceMissing
                except FileNotFoundError:
                    break
                except Exception as e:
                    raise e

In [55]:
signals_of_interest = ['FILT_TP9', 'FILT_AF7', 'FILT_AF8', 'FILT_TP10','HR_EMPATICA','EDA_EMPATICA','BVP_Filt']
rated_emotions = emotions[2:]

window_len, overlap_len = 30*256, 20*256
epoch_len = window_len - overlap_len
target_size = 256*115
all_sig_arr = np.zeros([1,target_size,len(signals_of_interest)])
all_labels_arr = np.zeros([1,len(rated_emotions)])
labels_arr_order = {e:i for i,e in enumerate(per_user_labels["22"]["BASELINE"].keys())}
subtracting_baseline = False

for user in per_user_stimulus_arr:
    user_data = per_user_stimulus_arr[user]
    for affect in user_data:
        if affect == "BASELINE":
            continue #continue onto the next loop

        #Get array of labels:
        labels = np.zeros([1,len(rated_emotions)])
        user_labels = per_user_labels[user]
        for e in rated_emotions:
            labels[0,labels_arr_order[e]] = user_labels[affect][e]
        if subtracting_baseline:
            pass

        all_labels_arr = np.vstack([all_labels_arr,labels])

        #Get signal arrays:

        data_chunk = user_data[affect][signals_of_interest].to_numpy()[256:target_size+256]
        data_len,num_chan = data_chunk.shape
        # print("data_len before: ", data_len)
        while data_len != target_size: #make all data the same by putting the beg of data at end
            data_chunk = np.vstack([data_chunk,data_chunk[:target_size-data_len]])
            data_len = data_chunk.shape[0]

        all_sig_arr = np.vstack([all_sig_arr,data_chunk[np.newaxis,:,:]])

#Epoching arrays:
split_arr = [all_sig_arr[1:,start:start + window_len,:]
             for start in range(0,data_len - window_len,epoch_len)]
num_splits = len(split_arr)
new_sig_arr = np.vstack(split_arr)
new_labels_arr = np.vstack([all_labels_arr[1:,:] for _ in range(num_splits)])

In [56]:
#Save data arrays:
np.save(r"C:\Users\nelso\Desktop\18065 Project Data\emognition_labels_arr.npy",new_labels_arr)
np.save(r"C:\Users\nelso\Desktop\18065 Project Data\emognition_signal_arr.npy",new_sig_arr)

In [57]:
#Load data array
processed_arr = np.load(r"C:\Users\nelso\Desktop\18065 Project Data\emognition_signal_arr.npy")
processed_labels = np.load(r"C:\Users\nelso\Desktop\18065 Project Data\emognition_labels_arr.npy")

In [58]:
X_train, X_test, y_train, y_test = train_test_split(processed_arr,processed_labels, test_size=0.2, random_state=42)

In [69]:
#Testing Data Integrity:
for _ in range(10):
    i = np.random.randint(0,len(X_train))
    print(f"Picking Data Point {i}")
    eeg_data = X_train[i,:,:-3]
    plt.figure()
    plt.plot([i/256 for i in range(len(eeg_data))],eeg_data)
    plt.title("EEG Spectrum Bands")
    bvp_data = X_train[i,:,np.array(signals_of_interest) == 'BVP_Filt'].flatten()
    print(bvp_data.shape)
    _, _, _, _, heart_rate = bvp(bvp_data,256,show=True)
    eda_data = X_train[i,:,np.array(signals_of_interest) == 'EDA_EMPATICA'].flatten()
    plt.figure()
    plt.plot([i/256 for i in range(len(eda_data))],eda_data)
    plt.title("EDA Data Plot")

Picking Data Point 390
(7680,)
Picking Data Point 10
(7680,)
Picking Data Point 309
(7680,)
Picking Data Point 1038
(7680,)
Picking Data Point 2257
(7680,)
Picking Data Point 861
(7680,)
Picking Data Point 2860
(7680,)


C:\Users\nelso\AppData\Local\Temp\ipykernel_36672\2266983084.py:13: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure()


Picking Data Point 1917
(7680,)
Picking Data Point 1470
(7680,)
Picking Data Point 386
(7680,)


In [70]:
X_train.shape

(2880, 7680, 7)

In [45]:
X_train.shape

(2880, 7680, 6)

In [46]:
y_train.shape

(2880, 9)

In [47]:
X_test.shape

(720, 7680, 6)

In [48]:
y_test.shape

(720, 9)

In [5]:
with open(r"C:\Users\nelso\Desktop\18065 Project Data\Emognition\22\22_BASELINE_STIMULUS_MUSE.json") as json_file:
    eeg_data = json.load(json_file)

In [13]:
for eeg_chan in ['FILT_TP9', 'FILT_AF7', 'FILT_AF8', 'FILT_TP10']:
    filtered_eeg = eeg_analysis(eeg_data)[0][eeg_chan]
    plt.figure()
    plt.plot([i/256 for i in range(len(eeg_df))],filtered_eeg)
    plt.title(f"EEG RAW Data {eeg_chan}")

filtered_eeg shape:  (30739,)
[ 35.15819602 189.76799187 431.44915043 ...  -1.8485881   -1.38779581
   1.24229346]
filtered_eeg shape:  (30739,)
[ 34.33975295 185.97486991 425.18208225 ...   6.48738635   3.89985449
   1.91576813]
filtered_eeg shape:  (30739,)
[ 3.46009575e+01  1.86950169e+02  4.25064290e+02 ... -6.36177604e+00
 -4.16509376e+00  7.10090294e-02]
filtered_eeg shape:  (30739,)
[ 34.89699146 188.70562466 430.05191096 ...  -8.08749227  -5.40383918
  -1.55377611]
filtered_eeg shape:  (30739,)
[ 35.15819602 189.76799187 431.44915043 ...  -1.8485881   -1.38779581
   1.24229346]
filtered_eeg shape:  (30739,)
[ 34.33975295 185.97486991 425.18208225 ...   6.48738635   3.89985449
   1.91576813]
filtered_eeg shape:  (30739,)
[ 3.46009575e+01  1.86950169e+02  4.25064290e+02 ... -6.36177604e+00
 -4.16509376e+00  7.10090294e-02]
filtered_eeg shape:  (30739,)
[ 34.89699146 188.70562466 430.05191096 ...  -8.08749227  -5.40383918
  -1.55377611]
filtered_eeg shape:  (30739,)
[ 35.15819602 

In [ ]:
eeg_df = pd.DataFrame.from_dict(eeg_data)
eeg_df.columns

In [348]:
for eeg_chan in ['RAW_TP9', 'RAW_AF7', 'RAW_AF8', 'RAW_TP10']:
    plt.figure()
    filtered_eeg = butter_bandpass_filter(eeg_df[eeg_chan], 5, 55, 256, order=4)
    plt.plot([i/256 for i in range(len(eeg_df))],filtered_eeg)
    plt.title(f"EEG RAW Data {eeg_chan}")

In [157]:
window = 30
seconds_of_data = window - 20 #window minus overlap
print("Training samples: ",2*60*256/(256*seconds_of_data)*64*0.9)
print("Training with augmentation: ", 2*2*60*256/(256*seconds_of_data)*64*0.9)
print("Testing samples: ",2*60*256/(256*seconds_of_data)*64*0.1)

Training samples:  691.2
Training with augmentation:  1382.4
Testing samples:  76.80000000000001


In [ ]:
#Calculations to ensure sampling rate and time of recordings match
approx_samp_rate_subject20 = 64*(30739/7680)
approx_exp_duration_subject20 = 30739/256/60

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Activation, Permute, Dropout
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, AveragePooling1D, MaxPooling1D
from tensorflow.keras.layers import SeparableConv2D, DepthwiseConv2D, DepthwiseConv1D, SeparableConv1D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import SpatialDropout2D
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.layers import Input, Flatten
from tensorflow.keras.constraints import max_norm
from tensorflow.keras import backend as K

In [ ]:
def SSVEPNet(nb_classes, Chans = 8, Samples = 256,
             dropoutRate = 0.5, kernLength = 64,
             D = 1, F2 = 16, norm_rate = 0.25, dropoutType = 'Dropout'):
    """
    SSVEPNet is a convolutional neural network that uses depthwise and separable
    convolutions to predict SSVEP signals

    nb_classes: int,number of classes that will be predicted
    Chans: int, the number of channels of EEG data
    Samples: int, the number of samples per EEG signal
    dropoutRate: foat, the dropout rate before the flattening layer
    kernelLength: int, the length of the 1D convolutions
    D: int, specifies the depth multiplier for first 1D convolution
    F2: int, number of filters in the second layer
    norm_rate: float, the normalization rate
    dropoutType: str, type of Dropout

    return: The Model built with tensorflow
    """

    if dropoutType == 'SpatialDropout2D':
        dropoutType = SpatialDropout2D
    elif dropoutType == 'Dropout':
        dropoutType = Dropout
    else:
        raise ValueError('dropoutType must be one of SpatialDropout2D '
                         'or Dropout, passed as a string.')

    input1   = Input(shape = (Samples, Chans))
    block1       = DepthwiseConv1D(kernLength, use_bias = False,
                                   depth_multiplier = D,padding='same')(input1)
    block2       = SeparableConv1D(F2,kernLength,
                                   use_bias = False, padding = 'same')(block1)
    block2       = BatchNormalization()(block2)
    block2       = Activation('elu')(block2)
    block2       = AveragePooling1D(4)(block2)
    block2       = SeparableConv1D(F2,int(kernLength/2),
                                   use_bias = False, padding = 'same')(block2)
    block2       = BatchNormalization()(block2)
    block2       = Activation('elu')(block2)
    block2       = AveragePooling1D(4)(block2)
    block2       = dropoutType(dropoutRate)(block2)

    flatten      = Flatten(name = 'flatten')(block2)

    dense        = Dense(nb_classes, name = 'dense',
                         kernel_constraint = max_norm(norm_rate))(flatten)
    softmax      = Activation('softmax', name = 'softmax')(dense)

    return Model(inputs=input1, outputs=softmax)